In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchvision.utils import make_grid

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

from tensorboardX import SummaryWriter

In [89]:
os.makedirs('images', exist_ok=True)

class Opt(object):
    def __init__(self):
        self.n_epochs = 200
        self.batch_size = 64
        self.lr = 0.0002
        self.b1 = 0.5
        self.b2 = 0.999
        self.n_cpu = 8
        self.latent_dim = 100
        self.img_size = 64
        self.channels = 1
        self.sample_interval = 100
        
opt = Opt()
# parser = argparse.ArgumentParser()
# parser.add_argument('--n_epochs', type=int, default=200, help='number of epochs of training')
# parser.add_argument('--batch_size', type=int, default=64, help='size of the batches')
# parser.add_argument('--lr', type=float, default=0.0002, help='adam: learning rate')
# parser.add_argument('--b1', type=float, default=0.5, help='adam: decay of first order momentum of gradient')
# parser.add_argument('--b2', type=float, default=0.999, help='adam: decay of first order momentum of gradient')
# parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
# parser.add_argument('--latent_dim', type=int, default=100, help='dimensionality of the latent space')
# parser.add_argument('--img_size', type=int, default=28, help='size of each image dimension')
# parser.add_argument('--channels', type=int, default=1, help='number of image channels')
# parser.add_argument('--sample_interval', type=int, default=400, help='interval betwen image samples')
# opt = parser.parse_args()
# print(opt)

In [90]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

In [105]:
class Generator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Generator, self).__init__()
        self.deconv1 = nn.ConvTranspose2d(100, d*8, 4, 1, 0)
        self.deconv1_bn = nn.BatchNorm2d(d*8)
        self.deconv2 = nn.ConvTranspose2d(d*8, d*4, 4, 2, 1)
        self.deconv2_bn = nn.BatchNorm2d(d*4)
        self.deconv3 = nn.ConvTranspose2d(d*4, d*2, 4, 2, 1)
        self.deconv3_bn = nn.BatchNorm2d(d*2)
        self.deconv4 = nn.ConvTranspose2d(d*2, d, 4, 2, 1)
        self.deconv4_bn = nn.BatchNorm2d(d)
        self.deconv5 = nn.ConvTranspose2d(d, 1, 4, 2, 1)

    # forward method
    def forward(self, input):
        x = F.relu(self.deconv1(input))
        x = F.relu(self.deconv1_bn(self.deconv1(input)))
        x = F.relu(self.deconv2_bn(self.deconv2(x)))
        x = F.relu(self.deconv3_bn(self.deconv3(x)))
        x = F.relu(self.deconv4_bn(self.deconv4(x)))
        x = F.tanh(self.deconv5(x))

        return x

In [106]:
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()

#         self.model = nn.Sequential(
#             nn.Linear(int(np.prod(img_shape)), 512),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 256),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(256, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, img):
#         img_flat = img.view(img.size(0), -1)
#         validity = self.model(img_flat)

#         return validity
    
class Discriminator(nn.Module):
    # initializers
    def __init__(self, d=128):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, d, 4, 2, 1)
        self.conv2 = nn.Conv2d(d, d*2, 4, 2, 1)
        self.conv2_bn = nn.BatchNorm2d(d*2)
        self.conv3 = nn.Conv2d(d*2, d*4, 4, 2, 1)
        self.conv3_bn = nn.BatchNorm2d(d*4)
        self.conv4 = nn.Conv2d(d*4, d*8, 4, 2, 1)
        self.conv4_bn = nn.BatchNorm2d(d*8)
        self.conv5 = nn.Conv2d(d*8, 1, 4, 1, 0)
        
    # forward method
    def forward(self, input):
        x = F.leaky_relu(self.conv1(input), 0.2)
        x = F.leaky_relu(self.conv2_bn(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.conv3_bn(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.conv4_bn(self.conv4(x)), 0.2)
        x = F.sigmoid(self.conv5(x))

        return x    

In [107]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

In [95]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [133]:
# Configure data loader
os.makedirs('../../data/mnist', exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.Scale(64),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=opt.batch_size, shuffle=True)

/home/takuma2460_0131/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/torchvision/transforms/transforms.py:187: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [135]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [149]:
for i, (imgs, _) in enumerate(dataloader):
    print("a")

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


KeyboardInterrupt: 

In [155]:
# ----------
#  Training
# ----------
opt.n_epochs = 100
writer = SummaryWriter()
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        # 1と0のラベルをbatch_size分だけ作成する
        valid = Variable(Tensor(imgs.size(0), 1, 1, 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1, 1, 1).fill_(0.0), requires_grad=False)
        
        # Configure input 
        # real_imgsのtypeをtensorに変換
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim, 1, 1))))

        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool the discriminator
        # discriminatorには1といって学習させる
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch, opt.n_epochs, i, len(dataloader),
                                                            d_loss.item(), g_loss.item()))

        x = make_grid(gen_imgs.data[:25], normalize=True, scale_each=True)
        writer.add_image('Image', x, i)
        writer.add_scalar("g_loss", g_loss.item(), i*epoch)
        writer.add_scalar("d_loss", d_loss.item(), i*epoch)
        
        batches_done = epoch * len(dataloader) + i
#         if batches_done % opt.sample_interval == 0:
#             save_image(gen_imgs.data[:25], 'images/%d.png' % batches_done, nrow=5, normalize=True)

/home/takuma2460_0131/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/takuma2460_0131/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[Epoch 0/100] [Batch 0/938] [D loss: 0.523147] [G loss: 0.635964]
[Epoch 0/100] [Batch 1/938] [D loss: 0.969898] [G loss: 3.880957]
[Epoch 0/100] [Batch 2/938] [D loss: 0.923864] [G loss: 0.260798]
[Epoch 0/100] [Batch 3/938] [D loss: 0.459459] [G loss: 3.229306]
[Epoch 0/100] [Batch 4/938] [D loss: 0.364646] [G loss: 1.172357]
[Epoch 0/100] [Batch 5/938] [D loss: 0.525784] [G loss: 0.954748]
[Epoch 0/100] [Batch 6/938] [D loss: 0.583977] [G loss: 1.738892]
[Epoch 0/100] [Batch 7/938] [D loss: 0.720323] [G loss: 0.432933]
[Epoch 0/100] [Batch 8/938] [D loss: 0.679327] [G loss: 2.694703]
[Epoch 0/100] [Batch 9/938] [D loss: 0.401460] [G loss: 1.022782]
[Epoch 0/100] [Batch 10/938] [D loss: 0.260115] [G loss: 1.896657]
[Epoch 0/100] [Batch 11/938] [D loss: 0.304410] [G loss: 1.547145]
[Epoch 0/100] [Batch 12/938] [D loss: 0.410694] [G loss: 1.527929]
[Epoch 0/100] [Batch 13/938] [D loss: 0.505490] [G loss: 0.719875]
[Epoch 0/100] [Batch 14/938] [D loss: 0.687703] [G loss: 2.303448]
[Epoc

[Epoch 0/100] [Batch 123/938] [D loss: 0.374726] [G loss: 4.110402]
[Epoch 0/100] [Batch 124/938] [D loss: 0.445136] [G loss: 0.680633]
[Epoch 0/100] [Batch 125/938] [D loss: 0.298794] [G loss: 3.921082]
[Epoch 0/100] [Batch 126/938] [D loss: 0.060161] [G loss: 3.508184]
[Epoch 0/100] [Batch 127/938] [D loss: 0.511605] [G loss: 0.635408]
[Epoch 0/100] [Batch 128/938] [D loss: 0.590527] [G loss: 4.038654]
[Epoch 0/100] [Batch 129/938] [D loss: 0.311121] [G loss: 1.219172]
[Epoch 0/100] [Batch 130/938] [D loss: 0.407693] [G loss: 1.000586]
[Epoch 0/100] [Batch 131/938] [D loss: 0.622804] [G loss: 3.468388]
[Epoch 0/100] [Batch 132/938] [D loss: 0.697546] [G loss: 0.431675]
[Epoch 0/100] [Batch 133/938] [D loss: 0.306493] [G loss: 2.901717]
[Epoch 0/100] [Batch 134/938] [D loss: 0.364427] [G loss: 2.232435]
[Epoch 0/100] [Batch 135/938] [D loss: 0.590149] [G loss: 0.526680]
[Epoch 0/100] [Batch 136/938] [D loss: 0.427059] [G loss: 2.185306]
[Epoch 0/100] [Batch 137/938] [D loss: 0.345576]

[Epoch 0/100] [Batch 244/938] [D loss: 0.239181] [G loss: 3.298580]
[Epoch 0/100] [Batch 245/938] [D loss: 0.350754] [G loss: 1.128303]
[Epoch 0/100] [Batch 246/938] [D loss: 0.339922] [G loss: 2.196011]
[Epoch 0/100] [Batch 247/938] [D loss: 0.269330] [G loss: 1.353807]
[Epoch 0/100] [Batch 248/938] [D loss: 0.255564] [G loss: 2.422520]
[Epoch 0/100] [Batch 249/938] [D loss: 0.294264] [G loss: 1.620167]
[Epoch 0/100] [Batch 250/938] [D loss: 0.271381] [G loss: 1.784716]
[Epoch 0/100] [Batch 251/938] [D loss: 0.389287] [G loss: 1.099103]
[Epoch 0/100] [Batch 252/938] [D loss: 0.668283] [G loss: 3.131779]
[Epoch 0/100] [Batch 253/938] [D loss: 1.058387] [G loss: 0.166339]
[Epoch 0/100] [Batch 254/938] [D loss: 0.803083] [G loss: 5.202615]
[Epoch 0/100] [Batch 255/938] [D loss: 0.207035] [G loss: 2.208984]
[Epoch 0/100] [Batch 256/938] [D loss: 0.587070] [G loss: 0.509594]
[Epoch 0/100] [Batch 257/938] [D loss: 0.360476] [G loss: 2.601809]
[Epoch 0/100] [Batch 258/938] [D loss: 0.470898]

[Epoch 0/100] [Batch 365/938] [D loss: 0.023852] [G loss: 3.519029]
[Epoch 0/100] [Batch 366/938] [D loss: 0.013827] [G loss: 4.641695]
[Epoch 0/100] [Batch 367/938] [D loss: 0.012683] [G loss: 4.737578]
[Epoch 0/100] [Batch 368/938] [D loss: 0.026014] [G loss: 3.741496]
[Epoch 0/100] [Batch 369/938] [D loss: 0.031569] [G loss: 3.480933]
[Epoch 0/100] [Batch 370/938] [D loss: 0.012521] [G loss: 5.241048]
[Epoch 0/100] [Batch 371/938] [D loss: 0.031071] [G loss: 5.712213]
[Epoch 0/100] [Batch 372/938] [D loss: 0.009004] [G loss: 4.729275]
[Epoch 0/100] [Batch 373/938] [D loss: 0.007625] [G loss: 5.148946]
[Epoch 0/100] [Batch 374/938] [D loss: 0.017984] [G loss: 3.777400]
[Epoch 0/100] [Batch 375/938] [D loss: 0.012396] [G loss: 4.258850]
[Epoch 0/100] [Batch 376/938] [D loss: 0.005411] [G loss: 5.627436]
[Epoch 0/100] [Batch 377/938] [D loss: 0.010040] [G loss: 5.553759]
[Epoch 0/100] [Batch 378/938] [D loss: 0.006923] [G loss: 5.161687]
[Epoch 0/100] [Batch 379/938] [D loss: 0.008732]

...

[Epoch 0/100] [Batch 849/938] [D loss: 0.605955] [G loss: 0.537698]
[Epoch 0/100] [Batch 850/938] [D loss: 0.298201] [G loss: 2.274061]
[Epoch 0/100] [Batch 851/938] [D loss: 0.209747] [G loss: 3.234059]
[Epoch 0/100] [Batch 852/938] [D loss: 0.286673] [G loss: 1.276900]
[Epoch 0/100] [Batch 853/938] [D loss: 0.216234] [G loss: 2.453211]
[Epoch 0/100] [Batch 854/938] [D loss: 0.212411] [G loss: 2.625686]
[Epoch 0/100] [Batch 855/938] [D loss: 0.280099] [G loss: 1.318964]
[Epoch 0/100] [Batch 856/938] [D loss: 0.309226] [G loss: 2.383149]
[Epoch 0/100] [Batch 857/938] [D loss: 0.305480] [G loss: 1.201545]
[Epoch 0/100] [Batch 858/938] [D loss: 0.363405] [G loss: 3.891180]
[Epoch 0/100] [Batch 859/938] [D loss: 0.201011] [G loss: 1.468822]
[Epoch 0/100] [Batch 860/938] [D loss: 0.103033] [G loss: 2.433273]
[Epoch 0/100] [Batch 861/938] [D loss: 0.167460] [G loss: 2.568099]
[Epoch 0/100] [Batch 862/938] [D loss: 0.280962] [G loss: 1.191191]
[Epoch 0/100] [Batch 863/938] [D loss: 0.703609]

[Epoch 1/100] [Batch 33/938] [D loss: 0.279038] [G loss: 1.116365]
[Epoch 1/100] [Batch 34/938] [D loss: 0.225967] [G loss: 1.730446]
[Epoch 1/100] [Batch 35/938] [D loss: 0.393592] [G loss: 3.787144]
[Epoch 1/100] [Batch 36/938] [D loss: 0.455405] [G loss: 0.671111]
[Epoch 1/100] [Batch 37/938] [D loss: 0.263580] [G loss: 3.662137]
[Epoch 1/100] [Batch 38/938] [D loss: 0.126558] [G loss: 2.433519]
[Epoch 1/100] [Batch 39/938] [D loss: 0.199548] [G loss: 2.024461]
[Epoch 1/100] [Batch 40/938] [D loss: 0.206126] [G loss: 1.624835]
[Epoch 1/100] [Batch 41/938] [D loss: 0.184581] [G loss: 3.361121]
[Epoch 1/100] [Batch 42/938] [D loss: 0.138105] [G loss: 1.932933]
[Epoch 1/100] [Batch 43/938] [D loss: 0.087107] [G loss: 3.721552]
[Epoch 1/100] [Batch 44/938] [D loss: 0.061237] [G loss: 4.220090]
[Epoch 1/100] [Batch 45/938] [D loss: 0.069023] [G loss: 2.889727]
[Epoch 1/100] [Batch 46/938] [D loss: 0.217337] [G loss: 1.344636]
[Epoch 1/100] [Batch 47/938] [D loss: 0.300275] [G loss: 4.823

[Epoch 1/100] [Batch 155/938] [D loss: 0.142891] [G loss: 2.725489]
[Epoch 1/100] [Batch 156/938] [D loss: 0.324596] [G loss: 2.242314]
[Epoch 1/100] [Batch 157/938] [D loss: 0.527389] [G loss: 0.579139]
[Epoch 1/100] [Batch 158/938] [D loss: 0.548541] [G loss: 4.108606]
[Epoch 1/100] [Batch 159/938] [D loss: 0.406155] [G loss: 0.989192]
[Epoch 1/100] [Batch 160/938] [D loss: 0.185158] [G loss: 2.760341]
[Epoch 1/100] [Batch 161/938] [D loss: 0.270115] [G loss: 1.831155]
[Epoch 1/100] [Batch 162/938] [D loss: 0.239521] [G loss: 1.853852]
[Epoch 1/100] [Batch 163/938] [D loss: 0.196648] [G loss: 2.065266]
[Epoch 1/100] [Batch 164/938] [D loss: 0.137650] [G loss: 2.461432]
[Epoch 1/100] [Batch 165/938] [D loss: 0.121354] [G loss: 3.214366]
[Epoch 1/100] [Batch 166/938] [D loss: 0.267977] [G loss: 1.390039]
[Epoch 1/100] [Batch 167/938] [D loss: 0.515161] [G loss: 4.217528]
[Epoch 1/100] [Batch 168/938] [D loss: 1.257160] [G loss: 0.124433]
[Epoch 1/100] [Batch 169/938] [D loss: 0.749105]

[Epoch 1/100] [Batch 276/938] [D loss: 0.300715] [G loss: 4.167032]
[Epoch 1/100] [Batch 277/938] [D loss: 0.175407] [G loss: 2.366552]
[Epoch 1/100] [Batch 278/938] [D loss: 0.209454] [G loss: 1.543098]
[Epoch 1/100] [Batch 279/938] [D loss: 0.269035] [G loss: 3.112659]
[Epoch 1/100] [Batch 280/938] [D loss: 0.437856] [G loss: 0.773426]
[Epoch 1/100] [Batch 281/938] [D loss: 0.365121] [G loss: 4.338806]
[Epoch 1/100] [Batch 282/938] [D loss: 0.205885] [G loss: 1.544822]
[Epoch 1/100] [Batch 283/938] [D loss: 0.176969] [G loss: 2.496137]
[Epoch 1/100] [Batch 284/938] [D loss: 0.151153] [G loss: 2.257500]
[Epoch 1/100] [Batch 285/938] [D loss: 0.169732] [G loss: 2.009007]
[Epoch 1/100] [Batch 286/938] [D loss: 0.090921] [G loss: 4.113435]
[Epoch 1/100] [Batch 287/938] [D loss: 0.206706] [G loss: 2.226676]
[Epoch 1/100] [Batch 288/938] [D loss: 0.202887] [G loss: 1.527756]
[Epoch 1/100] [Batch 289/938] [D loss: 0.162724] [G loss: 3.417068]
[Epoch 1/100] [Batch 290/938] [D loss: 0.190451]

KeyboardInterrupt: 